Load jupyter sql extension


In [1]:
#Load Jupter SQL extension
%reload_ext sql
%config SqlMagic.autocommit = False

In [2]:
#Load database
%sql sqlite:///../data/raw/frantamarit.db

'Connected: @../data/raw/frantamarit.db'

In [3]:
%%sql tables <<

SELECT 
    name
FROM 
    sqlite_master 
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%';

 * sqlite:///../data/raw/frantamarit.db
Done.
Returning data to local variable tables


In [24]:
%%sql master_join <<

SELECT
    *
FROM 
    rank_info
LEFT JOIN 
    personal_info
ON
    rank_info.id = personal_info.id
LEFT JOIN
    business_info
ON 
    rank_info.id = business_info.id
ORDER BY
    rank_info.position DESC;

 * sqlite:///../data/raw/frantamarit.db
Done.
Returning data to local variable master_join


In [25]:
#Showing tables
master_join

id,name,position,Unnamed: 0,id_1,lastName,age,Unnamed: 0_1,gender,country,image,id_2,Unnamed: 0_2,Source,worth,worthChange,realTimeWorth,realTimePosition
1690,OLGUN Zorlu,2208.0,2259,1690,zoRLU,54 years old,2259,Male,Turkey,https://specials-images.forbesimg.com/imageserve/064a602f59f8656c7e2a1765a579be37/416x416.jpg?background=000000&cropX1=0&cropX2=744&cropY1=219&cropY2=963,1690,2259,Diversified ==> diversified,1.0 BUSD,nan millions USD,None,1978
2599,zhuO Jun,2207.0,2258,2599,zhUO,52 years old,2258,F,Hong Kong,https://specials-images.forbesimg.com/imageserve/064a602f59f8656c7e2a1765a579be37/416x416.jpg?background=000000&cropX1=0&cropX2=744&cropY1=219&cropY2=963,2599,2258,Manufacturing ==> printed circuit boards,1.0 BUSD,0.0 millions USD,None,1978
8536,ZHU XINGMING,2206.0,2257,8536,zHU,51 years old,2257,None,People's Republic of China,https://specials-images.forbesimg.com/imageserve/064a602f59f8656c7e2a1765a579be37/416x416.jpg?background=000000&cropX1=0&cropX2=744&cropY1=219&cropY2=963,8536,2257,Manufacturing ==> electrical equipment,1.0 BUSD,0.0 millions USD,None,1856
5887,ZHOU LIANGZHANG,2205.0,2256,5887,zhOU,55 years old,2256,None,None,https://specials-images.forbesimg.com/imageserve/064a602f59f8656c7e2a1765a579be37/416x416.jpg?background=000000&cropX1=0&cropX2=744&cropY1=219&cropY2=963,5887,2256,Manufacturing ==> electrical equipment,1.0 BUSD,nan millions USD,None,2134
9216,zhao XIAOQIANg,2204.0,2255,9216,zHAO,51 years old,2255,M,China,https://specials-images.forbesimg.com/imageserve/064a602f59f8656c7e2a1765a579be37/416x416.jpg?background=000000&cropX1=0&cropX2=744&cropY1=219&cropY2=963,9216,2255,"Fashion & Retail ==> fashion, entertainment",1.0 BUSD,0.0 millions USD,None,2134
5372,YAO XInyi,2203.0,2254,5372,yAO,53 years old,2254,Male,None,https://specials-images.forbesimg.com/imageserve/064a602f59f8656c7e2a1765a579be37/416x416.jpg?background=000000&cropX1=0&cropX2=744&cropY1=219&cropY2=963,5372,2254,Diversified ==> air conditioning,1.0 BUSD,0.0 millions USD,None,2114
8677,yanG JIANXIn,2202.0,2253,8677,yANG,49 years old,2253,None,China,https://specials-images.forbesimg.com/imageserve/396ac9be937f8a7a801b375df40ec151/416x416.jpg?background=000000&cropX1=2&cropX2=744&cropY1=3&cropY2=746,8677,2253,Technology ==> apparel,1.0 BUSD,nan millions USD,None,2120
1831,wu chUNG-yi,2201.0,2252,1831,wU,59 years old,2252,None,None,https://specials-images.forbesimg.com/imageserve/396ac9be937f8a7a801b375df40ec151/416x416.jpg?background=000000&cropX1=2&cropX2=744&cropY1=3&cropY2=746,1831,2252,Manufacturing ==> manufacturing,1.0 BUSD,0.0 millions USD,None,2120
6273,WEN PENGCHENG,2200.0,2251,6273,WEN,55 years old,2251,M,None,https://specials-images.forbesimg.com/imageserve/5a90ae0ca7ea434a0c2723ca/416x416.jpg?background=000000&cropX1=347&cropX2=1212&cropY1=476&cropY2=1342,6273,2251,Food and Beverage ==> agribusiness,1.0 BUSD,nan millions USD,None,2144
7889,SANDY WEILL,2199.0,2250,7889,weILL,85 years old,2250,M,None,https://specials-images.forbesimg.com/imageserve/5a90ae0ca7ea434a0c2723ca/416x416.jpg?background=000000&cropX1=347&cropX2=1212&cropY1=476&cropY2=1342,7889,2250,Finance and Investments ==> Citigroup,1.0 BUSD,0.0 millions USD,None,2144


1. Creating engine to connect to the database

In [5]:
#from sqlalchemy import create_engine
import pandas as pd
from sqlalchemy import create_engine

2. Create engine and get data tables

In [7]:
#Creating engine
eng = create_engine('sqlite:///../data/raw/frantamarit.db')

3. Creating data frames for each table

In [8]:
#Creating function to ingest sql table and convert it to dataframe
def create_df_sql(x):
    return pd.read_sql_table(x, con=eng)

#Defining dataframes
df_business_info = create_df_sql('business_info')
df_personal_info = create_df_sql('personal_info')
df_rank_info = create_df_sql('rank_info')

In [9]:
def outer_merge(x, y, key):
    return pd.merge(x, y, on=key, how='outer')

In [10]:
df_partial_master = outer_merge(df_rank_info,df_personal_info,'id')

In [16]:
df_master =  outer_merge(df_partial_master, df_business_info, 'id')

In [17]:
df_master.sort_values('position',ascending=False)

,id,name,position,Unnamed: 0_x,lastName,age,Unnamed: 0_y,gender,country,image,Unnamed: 0,Source,worth,worthChange,realTimeWorth,realTimePosition
2259,1690,OLGUN Zorlu,2208.0,2259,zoRLU,54 years old,2259.0,Male,Turkey,https://specials-images.forbesimg.com/imageser...,2259.0,Diversified ==> diversified,1.0 BUSD,nan millions USD,NaN,1978.0
2258,2599,zhuO Jun,2207.0,2258,zhUO,52 years old,2258.0,F,Hong Kong,https://specials-images.forbesimg.com/imageser...,2258.0,Manufacturing ==> printed circuit boards,1.0 BUSD,0.0 millions USD,NaN,1978.0
2257,8536,ZHU XINGMING,2206.0,2257,zHU,51 years old,2257.0,None,People's Republic of China,https://specials-images.forbesimg.com/imageser...,2257.0,Manufacturing ==> electrical equipment,1.0 BUSD,0.0 millions USD,NaN,1856.0
2256,5887,ZHOU LIANGZHANG,2205.0,2256,zhOU,55 years old,2256.0,None,None,https://specials-images.forbesimg.com/imageser...,2256.0,Manufacturing ==> electrical equipment,1.0 BUSD,nan millions USD,NaN,2134.0
2255,9216,zhao XIAOQIANg,2204.0,2255,zHAO,51 years old,2255.0,M,China,https://specials-images.forbesimg.com/imageser...,2255.0,"Fashion & Retail ==> fashion, entertainment",1.0 BUSD,0.0 millions USD,NaN,2134.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47,8911,supaluck umPUJH & Family,NaN,47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,4098,thomaS Duff,NaN,48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,6885,zakhar SMUSHkin,NaN,49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,5401,zhang ZHONGNENG & family,NaN,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
#Quick view of rank_info
df_personal_info.head()

,id,lastName,age,Unnamed: 0,gender,country,image
0,6528,beZOS,54 years old,52,M,None,https://specials-images.forbesimg.com/imageser...
1,5875,gaTES,62 years old,53,None,United States,https://specials-images.forbesimg.com/imageser...
2,8760,buFFEtt,87 years old,54,None,United States,https://specials-images.forbesimg.com/imageser...
3,4685,aRNAULT,69 years old,55,M,None,https://specials-images.forbesimg.com/imageser...
4,8584,zUCKERBERG,1985,56,M,United States,https://specials-images.forbesimg.com/imageser...


In [14]:
#Get unique values of column Unnamed: 0
df_rank_info['Unnamed: 0'].unique()

array([   0,    1,    2, ..., 2257, 2258, 2259])

In [15]:
#Looks like the column is redundant, so we will drop it
def drop_col_in(x,y):
    return x.drop(y, axis=1, inplace=True)

drop_col_in(df_rank_info, 'Unnamed: 0')

In [16]:
#Feature name is not written correctly, so let's harmonized it and lower case
#the name.
def lowercase_feature(x,y):
    x[y] = x[y].apply(lambda z: z.lower())
    return x[y]

lowercase_feature(df_rank_info, 'name')

0       anant asavabhokhin & family
1                    arash ferdowsi
2          banthoon lamsam & family
3            bill adderley & family
4                 boris zingarevich
                   ...             
2255                 zhao xiaoqiang
2256                zhou liangzhang
2257                   zhu xingming
2258                       zhuo jun
2259                    olgun zorlu
Name: name, Length: 2260, dtype: object

In [20]:
#We don't want to get rid of the NaN yet, so we'll fill them with 0s.
def fill_0(x):
    return x.fillna(0, inplace=True)

fill_0(df_rank_info)
df_rank_info

,id,name,position
0,5542,anant asavabhokhin & family,0.0
1,4183,arash ferdowsi,0.0
2,4761,banthoon lamsam & family,0.0
3,1449,bill adderley & family,0.0
4,1624,boris zingarevich,0.0
...,...,...,...
2255,9216,zhao xiaoqiang,2204.0
2256,5887,zhou liangzhang,2205.0
2257,8536,zhu xingming,2206.0
2258,2599,zhuo jun,2207.0


In [17]:
#get nulls

def get_null(x):
    return x.isnull().sum()

In [21]:
get_null(df_rank_info)

id          0
name        0
position    0
dtype: int64

In [32]:
len(df_rank_info)
#df_rank_info['Unnamed: 0'].le

2260

In [38]:
print(rank_info.notnull().sum())
print(personal_info.notnull().sum())

NameError: name 'rank_info' is not defined

In [ ]:
#Formula to drop column inplace
def drop_col_in(x,y):
    return x.drop(y, axis=1, inplace=True)

drop_col_in()

"""
rank_info.drop('Unnamed: 0', axis=1, inplace=True)
personal_info.drop('Unnamed: 0', axis=1, inplace=True)
business_info.drop('Unnamed: 0', axis=1, inplace=True)
"""

In [ ]:
personal_info.head()

In [ ]:
partial_info = pd.merge(rank_info, personal_info, on='id', how='inner')
master_info = pd.merge(partial_info, business_info, on='id', how='inner')
master_info.head()


In [ ]:
#Cleaning and lowercasing name and lastName features
master_info[['name','lastName']] = \
master_info[['name','lastName']].apply(lambda x: x.astype(str).str.lower())

In [ ]:
master_info.isnull().sum()

In [ ]:
#Renaming feature names
master_info.rename(index={e: e+1 for e in range(len(master_info))}, 
                columns={'name': 'full_name', 
                         'lastName': 'last_name',
                        'Source': 'source'}, 
                inplace=True)

In [ ]:
master_info.head()